# Практическое задание к уроку "Uplift-моделирование"

## Импорт библиотек

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

## Задание 1

скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [44]:
campaings = pd.read_csv('./data.csv')
campaings.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


## Задание 2

Поле conversion - это целевая переменная, а offer - коммуникация.  
Переименовать поля (conversion -> target, offer -> treatment)  
Привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [45]:
# Переименовать поля (conversion -> target, offer -> treatment) 
campaings.rename(columns={"conversion": "target", "offer": "treatment"}, inplace=True)
campaings.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


In [48]:
# Привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
if campaings["treatment"].dtype != np.int64:
    campaings["treatment"] = campaings["treatment"].map(lambda x: int(x != "No Offer"))
campaings["treatment"].head(3)

0    1
1    0
2    1
Name: treatment, dtype: int64

## Задание 3

сделать разбиение набора данных на тренировочную и тестовую выборки

In [49]:
features = campaings.drop(columns=["target"])
target = campaings["target"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    test_size=0.33, 
    random_state=RANDOM_STATE
)

## Задание 4

сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

## Задание 5
 
провести uplift-моделирование 3 способами: 

1. модель с признаком коммуникации (S learner)
2. модель с трансформацией таргета (трансформация классов п. 2. 1) 
3. вариант с двумя независимыми моделями

## Задание 6

в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей


## Задание 7

построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

### Ссылки

1. https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0
2. https://habr.com/ru/company/ru_mts/blog/485980/#reference1
3. https://en.wikipedia.org/wiki/Uplift_modelling
4. https://www.youtube.com/watch?v=yFQAIJBYXI0
5. https://www.youtube.com/watch?v=jCUcYiBK03I